In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from joblib import load
from tqdm import tqdm
import warnings

from sklearn.metrics import classification_report
warnings.filterwarnings("ignore")
import glob
import os
from collections import OrderedDict
from src.layers.classifiers import TokenClassifier, MixedClassifier

import h5py
import sys
import umap
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib import cm

In [2]:
dict = {
    "AGN": 0,
    "QSO": 1,
    "EA": 2,
    "YSO": 3,
    "SNIa": 4,
    "CV/Nova": 5,
    "RRLc": 6,
    "RSCVn": 7,
    "Blazar": 8,
    "SNII": 9,
    "EB/EW": 10,
    "LPV": 11,
    "CEP": 12,
    "RRLab": 13,
    "Periodic-Other": 14,
    "DSCT": 15,
    "SNIbc": 16,
    "SLSN": 17,
    "TDE": 18,
    "SNIIb": 19,
    "SNIIn": 20,
    "Microlensing": 21
}

In [5]:

path = '/home/mdelafuente/'#sys.argv[2]

device = torch.device(f"cuda:2" if torch.cuda.is_available() else "cpu")


abs_path = '//home/mdelafuente/pipeline/pipeline/training/lc_classifier_ztf/ATAT_ALeRCE/results/ZTF_ff/LC/pretrain/'
path_args = glob.glob(f"{abs_path}*args*")[0]
print(path_args)
import yaml
with open(path_args, 'r') as file:
    args = yaml.safe_load(file)

//home/mdelafuente/pipeline/pipeline/training/lc_classifier_ztf/ATAT_ALeRCE/results/ZTF_ff/LC/pretrain/args.yaml


In [7]:
print('Initializing models') 
from src.layers.cATAT import LightCurveClassifier
model = LightCurveClassifier(**args) 


print('Loading checkpoints')
checkpoint_path_clip = glob.glob(
    f"{abs_path}*my_best_checkpoint*"
) 
print(checkpoint_path_clip[-1])
checkpoint_clip = torch.load(checkpoint_path_clip[-1], map_location=torch.device('cuda:2'))
od_atat = OrderedDict() 

for key in checkpoint_clip["state_dict"].keys(): 
        #print(key)
         
        if 'model' in key:
            od_atat[key.replace("model.", "")] = checkpoint_clip["state_dict"][key]

model.load_state_dict(od_atat,strict=True)

Initializing models
Loading checkpoints
//home/mdelafuente/pipeline/pipeline/training/lc_classifier_ztf/ATAT_ALeRCE/results/ZTF_ff/LC/pretrain/my_best_checkpoint-step=36289.ckpt


RuntimeError: Error(s) in loading state_dict for LightCurveClassifier:
	Missing key(s) in state_dict: "LC.time_encoder.time_encoders.0.alpha_sin", "LC.time_encoder.time_encoders.0.alpha_cos", "LC.time_encoder.time_encoders.0.beta_sin", "LC.time_encoder.time_encoders.0.beta_cos", "LC.time_encoder.time_encoders.0.ar", "LC.time_encoder.time_encoders.0.linear_proj.0.weight", "LC.time_encoder.time_encoders.0.linear_proj.0.bias", "LC.time_encoder.time_encoders.1.alpha_sin", "LC.time_encoder.time_encoders.1.alpha_cos", "LC.time_encoder.time_encoders.1.beta_sin", "LC.time_encoder.time_encoders.1.beta_cos", "LC.time_encoder.time_encoders.1.ar", "LC.time_encoder.time_encoders.1.linear_proj.0.weight", "LC.time_encoder.time_encoders.1.linear_proj.0.bias", "LC.transformer_lc.stacked_transformers.0.layer_norm.0.a_2", "LC.transformer_lc.stacked_transformers.0.layer_norm.0.b_2", "LC.transformer_lc.stacked_transformers.0.layer_norm.1.a_2", "LC.transformer_lc.stacked_transformers.0.layer_norm.1.b_2", "L

In [ ]:
seed = 0
from src.data.modules.LitData  import LitData
args['general']['data_root'] = 'data/datasets/ZTF_ff/final/LC_MD_FEAT_240627_windows_200_12'
args['general']['data_root'] = 'data/datasets/ZTF_ff/final/LC_MD_FEAT_240627_windows_200_12'
args['general']['use_sampler'] = False
pl_datal = LitData(**args['general'])
dataloader = pl_datal.val_dataloader() 
    

In [ ]:
target = None
preds_out = None

model.eval().to(device = 'cuda:2')  

for b1 in tqdm(dataloader):

    #print(b1.keys())
    b1 = {k: b1[k].float().to(device = 'cuda:2') for k in b1.keys()}

    lc_emb = model(**b1)  
   
    t = b1['labels']


    preds_out = (
        np.concatenate([preds_out, torch.argmax(lc_emb, axis=1).cpu().numpy()])
        if preds_out is not None
        else torch.argmax(lc_emb, axis=1).cpu().detach().numpy()
    )

    target = (
        np.concatenate([target, t.cpu().detach().numpy()])
        if target is not None
        else t.cpu().numpy()
    )

In [ ]:
results = preds_out
classification = classification_report(target, results, target_names=list(dict.keys()),digits = 4)
print(classification)

In [ ]:
import matplotlib.pyplot as plt
from utils.plottinglib import elasticc_confusion_matrix

out_metrics_balto = classification_report(
    target, results, target_names=list(dict.keys()), output_dict=True
)["macro avg"]

template_balto = ""

for key in out_metrics_balto.keys():
    template_balto += " {} : {:.3f} ".format(key.upper(), out_metrics_balto[key])
fig, axes = plt.subplots(1, 1, figsize=(12, 12))
elasticc_confusion_matrix(
    y_true=target,
    y_pred=results,
    classes= np.array(list(dict.keys())),
    ax=axes,
    normalize=True,
    title=f" FClassifier Results [TEST] \n\n {template_balto}",
)